In [1]:
suppressMessages(library(miloR))
suppressMessages(library(SingleCellExperiment))
suppressMessages(library(scater))
suppressMessages(library(dplyr))
suppressMessages(library(patchwork))
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ tibble  3.1.8     ✔ purrr   0.3.4
✔ tidyr   1.2.1     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::collapse()   masks IRanges::collapse()
✖ dplyr::combine()    masks Biobase::combine(), BiocGenerics::combine()
✖ dplyr::count()      masks matrixStats::count()
✖ dplyr::desc()       masks IRanges::desc()
✖ tidyr::expand()     masks S4Vectors::expand()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::first()      masks S4Vectors::first()
✖ dplyr::lag()        masks stats::lag()
✖ ggplot2::Position() masks BiocGenerics::Position(), base::Position()
✖ purrr::reduce()     masks GenomicRanges::reduce(), IRanges::reduce()
✖ dplyr::rename()     masks S4Vectors::rename()
✖ dplyr::slice()      masks IRanges::slice()


In [19]:
source("milo.R")
source("../00_preprocessing/preprocessing.R")

Attaching SeuratObject

Attaching sp


Attaching package: ‘Seurat’


The following object is masked from ‘package:SummarizedExperiment’:

    Assays




In [ ]:
milo_obj = qs::qread('_targets/objects/milo_Astrocytes')
da_results = qs::qread('_targets/objects/da_results_01_Astrocytes___obob5v5')

In [ ]:
# a wide df with the log fc of every nh
# 1 row
# nhood cols
make_logFC_df = function(da_results){
    logFC_df = da_results %>% 
        arrange(Nhood) %>% select(logFC) %>% t %>% as.data.frame
    logFC_df 
}

In [ ]:
# a df of nh cols
# by cell (barcode) rows
make_nhm = function(milo_obj){
    nh = milo_obj@nhoods
    nh@Dimnames[[2]] = seq(1:length(nh@Dimnames[[2]]))
    nhm = nh %>% as.matrix %>% as.data.frame
    nhm
}

In [ ]:
annotate_nhood_counts = function(da_results, nhm){
    nhood_counts = nhm %>% colSums %>% enframe %>%
      transmute(Nhood=name, n_cells=value) %>%
      mutate(Nhood = as.double(Nhood))
    da_results = dplyr::left_join(x=da_results, y=nhood_counts)
    da_results
}

In [ ]:
make_nhood_summary = function(da_results, nhm){
    nhood_summary = da_results %>% 
        # filter(SpatialFDR < 0.1) %>%
        group_by(polarity) %>%
        summarise(n_cells = sum(n_cells),
                  n_nhoods = n()) %>%
        mutate(cell_freq = n_cells/sum(n_cells),
               nhood_freq = n_nhoods/sum(n_nhoods)) %>%
        mutate(n_cells_nhood = n_cells)
    nhood_summary = nhood_summary %>%
        mutate(n_cells = round(dim(nhm)[1] * cell_freq))
    nhood_summary
}

In [ ]:
check_split_validity = function(nhood_summary, min_cluster_frac=0.025, min_cells=200){
    nhood_summary = nhood_summary %>%
        mutate(split_valid = case_when((cell_freq >= min_cluster_frac & n_cells >= min_cells) ~ TRUE,
                                       TRUE ~ FALSE)) %>%
        mutate(adj_split = NA)
    if (dim(nhood_summary)[1] > 1){
        nhood_summary = nhood_summary %>% 
        mutate(adj_split = case_when(polarity == 'neg' ~ "none",
                                     polarity == 'pos' ~ "none",
                                     polarity == 'none' ~ 'huh'))
        if (nhood_summary %>% pull(adj_split) %>% `==`('huh') %>% any()){
            # this gets the smallest adj split to none, if 'none' is too small
            pos_or_neg = nhood_summary %>% filter(polarity != 'none') %>% arrange(n_cells) %>% pull(polarity) %>% `[[`(1)
            nhood_summary = nhood_summary %>% 
              mutate(adj_split = case_when(adj_split == "huh" ~ pos_or_neg,
                                           TRUE ~ adj_split))
        }
    }
    nhood_summary
}

In [ ]:
combine_splits = function(da_results, nhm, min_cluster_frac=0.025, min_cells=100){
    while (TRUE){
        nhood_summary = make_nhood_summary(da_results, nhm) %>%
          check_split_validity(min_cluster_frac=0.025, min_cells=200)
        if (dim(nhood_summary)[1] == 1){
            # already combined!
            return(da_results)
        } else if (nhood_summary %>% pull(split_valid) %>% all()) {
            # all splits valid
            return(da_results)
        } else {
            # merges the smallest cluster
            split_to_merge = nhood_summary %>% 
                filter(split_valid == FALSE) %>%
                arrange(n_cells) %>%
                slice(1) 
            old_polarity = split_to_merge$polarity
            new_polarity = split_to_merge$adj_split
            da_results = da_results %>%
                mutate(polarity = case_when(polarity == old_polarity ~ new_polarity,
                                            TRUE ~ polarity))
        }
    }
}

In [ ]:
make_logFC_cells_df = function(nhm, logFC_df){
    logFC_cells = mapply(`*`, nhm, (logFC_df)) %>% as.data.frame
    rownames(logFC_cells) = rownames(nhm)
    logFC_cells
}

In [ ]:
nonzero_mean = function(x, na.rm=TRUE, zero.propagate = FALSE){
  x = x[x != 0]
  if (length(x) > 0){
   nzm = mean(x)
  } else {
  nzm = 0
  }
  nzm
}

In [ ]:
make_nzdf = function(logFC_cells){
    nonzero_mean_df = logFC_cells %>% rowwise() %>%
    summarise(nonzero_mean = nonzero_mean(c_across(everything())),
              nonzero_sum = sum(c_across(everything())))
    nonzero_mean_df$rownames = rownames(logFC_cells)
    nonzero_mean_df = nonzero_mean_df %>% as.data.frame %>% column_to_rownames(var = 'rownames')
    nonzero_mean_df = nonzero_mean_df %>% arrange(nonzero_sum)
    nonzero_mean_df
}


In [ ]:
obj = qs::qread('_targets/objects/obj_Astrocytes')
obj

In [ ]:
get_cell_freq = function(nhood_summary, split_name){
  cell_freq = ifelse(nhood_summary %>% filter(polarity == split_name) %>% nrow %>% `>`(0),
                     nhood_summary %>% filter(polarity == split_name) %>% pull(cell_freq),
                     as.double(0))
  cell_freq
}

make_barcode_polarity_df = function(nonzero_mean_df, nhood_summary){
    if (nhood_summary %>% filter(polarity == 'neg') %>% dim %>% `[`(1) %>% `>`(0)){
        neg_cells = top_frac(nonzero_mean_df, -get_cell_freq(nhood_summary, 'neg'), wt=nonzero_sum) %>% 
                      rownames %>% 
                      data.frame(barcodes=., polarity=rep('neg', length(.)))
    } else {
        neg_cells = data.frame(barcodes=character(), polarity=character())
    }
    if (nhood_summary %>% filter(polarity == 'pos') %>% dim %>% `[`(1) %>% `>`(0)){
        pos_cells = top_frac(nonzero_mean_df, get_cell_freq(nhood_summary, 'pos'), wt=nonzero_sum) %>% 
                      rownames %>% 
                      data.frame(barcodes=., polarity=rep('pos', length(.)))
    } else {
        pos_cells = data.frame(barcodes=character(), polarity=character())
    }
    none_cells = setdiff(rownames(nonzero_mean_df), union(neg_cells$barcodes, pos_cells$barcodes)) %>%
                 data.frame(barcodes=., polarity=rep('none', length(.)))
    barcode_df = rbind(neg_cells, none_cells, pos_cells)
    barcode_df
}

In [ ]:
logFC_df = make_logFC_df(da_results)
nhm = make_nhm(milo_obj)
logFC_cells = make_logFC_cells_df(nhm, logFC_df)
da_results = annotate_nhood_counts(da_results, nhm)
da_results = combine_splits(da_results, nhm)
nonzero_mean_df = make_nzdf(logFC_cells)
nhood_summary = make_nhood_summary(da_results, nhm) %>% check_split_validity()
barcode_df = make_nhood_summary(da_results, nhm) %>% check_split_validity() %>%
             make_barcode_polarity_df(nonzero_mean_df, .)


In [ ]:
head(barcode_df)

In [ ]:
make_pldf = function(barcode_df, milo_obj){
    labels_df = milo_obj %>% `@`('colData') %>% data.frame %>% select(labels) %>% rownames_to_column(var = "barcodes")
    pldf = left_join(labels_df, barcode_df) %>% 
        mutate(polar_label = paste0(labels, '.', polarity)) %>%
        select(-c(labels, polarity)) %>%
        mutate(barcodes =  factor(barcodes, levels = labels_df$barcodes)) %>%
        column_to_rownames("barcodes")
    pldf
}

In [ ]:
labels_df = milo_obj %>% `@`('colData') %>% data.frame %>% select(labels) %>% rownames_to_column(var = "barcodes")
head(labels_df)

In [ ]:
labels_df = milo_obj %>% `@`('colData') %>% data.frame %>% select(labels) %>% rownames_to_column(var = "barcodes")
dim(labels_df)

In [ ]:
dim(barcode_df)

In [ ]:
barcode_df %>% group_by(barcodes) %>% filter(n()>1)

In [ ]:
left_join(labels_df, barcode_df) %>% 
    mutate(polar_label = paste0(labels, '.', polarity)) %>%
    select(-c(labels, polarity)) %>%
    mutate(barcodes =  factor(barcodes, levels = labels_df$barcodes)) %>%
    column_to_rownames("barcodes")

In [ ]:
a$barcodes

In [ ]:
rbind(enframe(neg_cells), enframe(none_cells), enframe(pos_cells))

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
DimPlot(obj, reduction = "umap", cells.highlight = neg_cells, label = TRUE,  repel = TRUE)

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
DimPlot(obj, reduction = "umap", cells.highlight = pos_cells, label = TRUE,  repel = TRUE)

In [ ]:
obj_other = qs::qread('../00_preprocessing/_targets/objects/exp_labelled_other')
obj_other

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
DimPlot(obj_other, reduction = "umap", cells.highlight = neg_cells, label = TRUE,  repel = TRUE)

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
DimPlot(obj_other, reduction = "umap", cells.highlight = pos_cells, label = TRUE,  repel = TRUE)

In [13]:
da_results = qs::qread('_targets/objects/da_results_02_NFOL___obob5v5')
head(da_results)

,logFC,logCPM,F,PValue,FDR,Nhood,SpatialFDR,polarity,labels,labels_fraction,n_cells
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
1,3.760794,16.60684,19.35516,1.284295e-05,0.0002873421,8,0.0002575553,pos,NFOL,1,130
2,3.491975,16.46112,18.12992,2.394517e-05,0.0002873421,18,0.0002575553,pos,NFOL,1,109
3,3.030633,16.71710,15.66998,8.444858e-05,0.0005066915,6,0.0005158622,pos,NFOL,1,141
4,-2.411453,16.48335,15.91158,7.457008e-05,0.0005066915,12,0.0005158622,pos,NFOL,1,69
5,3.383601,16.39988,14.79811,1.324624e-04,0.0006063572,3,0.0006294792,pos,NFOL,1,98
6,2.905064,16.64919,14.22507,1.782626e-04,0.0006111862,9,0.0006488172,pos,NFOL,1,122


In [71]:
da_results$NhoodGroup = da_results$polarity
da_results

logFC,logCPM,F,PValue,FDR,Nhood,SpatialFDR,polarity,labels,labels_fraction,n_cells,NhoodGroup
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>
3.76079425,16.60684,1.935516e+01,1.284295e-05,0.0002873421,8,0.0002575553,pos,NFOL,1,130,pos
3.49197478,16.46112,1.812992e+01,2.394517e-05,0.0002873421,18,0.0002575553,pos,NFOL,1,109,pos
3.03063326,16.71710,1.566998e+01,8.444858e-05,0.0005066915,6,0.0005158622,pos,NFOL,1,141,pos
-2.41145251,16.48335,1.591158e+01,7.457008e-05,0.0005066915,12,0.0005158622,pos,NFOL,1,69,pos
3.38360083,16.39988,1.479811e+01,1.324624e-04,0.0006063572,3,0.0006294792,pos,NFOL,1,98,pos
2.90506352,16.64919,1.422507e+01,1.782626e-04,0.0006111862,9,0.0006488172,pos,NFOL,1,122,pos
-2.82397977,16.33794,1.453763e+01,1.515893e-04,0.0006063572,21,0.0006488172,pos,NFOL,1,56,pos
2.82651185,16.77029,1.357727e+01,2.496512e-04,0.0007489535,1,0.0007753989,pos,NFOL,1,149,pos
2.52799766,16.49609,1.320451e+01,3.032117e-04,0.0008085645,2,0.0008181182,pos,NFOL,1,107,pos


In [14]:
da_results %>% pull(polarity) %>% unique

[1] "pos"

In [ ]:
qs::qread('_targets/objects/')

In [72]:
milo_obj = qs::qread('_targets/objects/milo_MOL')

In [8]:
milo_obj

class: Milo 
dim: 17447 4023 
metadata(0):
assays(2): counts logcounts
rownames(17447): Lypla1 Tcea1 ... ENSMUSG00000095742 ENSMUSG00000095041
rowData names(0):
colnames(4023): ATTACTCTCCGCGGAT_1_1 AGTTCGAAGAAACCAT_1_1 ...
  AGGCATTGTCATCGGC_5_3 TGTAGACCAATGAAAC_5_3
colData names(36): orig.ident nCount_RNA ... group ident
reducedDimNames(2): PCA UMAP
mainExpName: SCT
altExpNames(0):
nhoods dimensions(2): 4023 189
nhoodCounts dimensions(2): 189 36
nhoodDistances dimension(1): 189
graph names(1): graph
nhoodIndex names(1): 189
nhoodExpression dimension(2): 1 1
nhoodReducedDim names(0):
nhoodGraph names(1): nhoodGraph
nhoodAdjacency dimension(2): 189 189

In [17]:
milo_obj@assays@data$logcounts %>% rownames %in% ('Lypla1')

[1]  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
   [13] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
   [25] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
   [37] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
   [49] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
   [61] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
   [73] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
   [85] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
   [97] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [109] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [121] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [133] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [145] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [157] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [169] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [181] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [193] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [205] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [217] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [229] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [241] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [253] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [265] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [277] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [289] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [301] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [313] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [325] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [337] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [349] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [361] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [373] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [385] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [397] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [409] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [421] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [433] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [445] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [457] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [469] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [481] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [493] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [505] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [517] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [529] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [541] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [553] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [565] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [577] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [589] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [6

In [41]:
.get_only_named_genes_vec = function(vec_of_genes){
    not_a_rik = vec_of_genes %>% stringr::str_ends("Rik") %>% `!`
    not_an_ensmusg = vec_of_genes %>% stringr::str_starts("ENSMUSG") %>% `!`
    not_a_Gm = vec_of_genes %>% stringr::str_starts("Gm\\d") %>% `!`
    not_4_or_more_digits = vec_of_genes %>%  stringr::str_ends("\\d{4,}") %>% `!`
    only_named_genes = (not_a_rik & not_an_ensmusg & not_a_Gm & not_4_or_more_digits) %>% which
    only_named_genes
}

.make_dgc_ngo = function(dgcm){
    ngo_genes_idx = dgcm@Dimnames[[1]] %>% .get_only_named_genes_vec
    dgcm = dgcm[ngo_genes_idx,]
    dgcm
}

make_milo_ngo = function(milo_obj){
    milo_obj@assays@data$logcounts = .make_dgc_ngo(milo_obj@assays@data$logcounts)
    milo_obj@assays@data$counts = .make_dgc_ngo(milo_obj@assays@data$counts)
    milo_obj
}

In [73]:
ngo_genes_idx = milo_obj@assays@data$logcounts@Dimnames[[1]] %>% .get_only_named_genes_vec

In [76]:
milo_obj@assays@data$logcounts@Dimnames[[1]][ngo_genes_idx]

[1] "Lypla1"         "Tcea1"          "Atp6v1h"        "Oprk1"         
    [5] "Rb1cc1"         "St18"           "Pcmtd1"         "Rrs1"          
    [9] "Adhfe1"         "Sgk3"           "Mcmdc2"         "Cspp1"         
   [13] "Prex2"          "Sulf1"          "Kcnb2"          "Terf1"         
   [17] "Rdh10"          "Tmem70"         "Ly96"           "Gdap1"         
   [21] "Crispld1"       "Tfap2d"         "Paqr8"          "Efhc1"         
   [25] "Tmem14a"        "Gsta3"          "B3gat2"         "Col9a1"        
   [29] "Lmbrd1"         "Khdrbs2"        "Rab23"          "Dst"           
   [33] "Imp4"           "Ptpn18"         "Amer3"          "Arhgef4"       
   [37] "Plekhb2"        "Hs6st1"         "Arid5a"         "Fer1l5"        
   [41] "Cnnm4"          "Cnnm3"          "Cox5b"          "Vwa3b"         
   [45] "Cnga3"          "Inpp4a"         "Unc50"          "Lipt1"         
   [49] "Mrpl30"         "Eif5b"          "Pdcl3"          "Npas2"         
   [53] "Rpl31"          "Cnot11"         "Map4k4"         "Il1r2"         
   [57] "Il1r1"          "Il1rl2"         "Il1rl1"         "Slc9a2"        
   [61] "Mrps9"          "Gpr45"          "Nck2"           "Ecrg4"         
   [65] "Tpp2"           "Bivm"           "Ercc5"          "Gulp1"         
   [69] "Wdr75"          "Dnah7b"         "Dnah7c"         "Tmeff2"        
   [73] "Stat4"          "Stat1"          "Nemp2"          "Hibch"         
   [77] "Ormdl1"         "Osgepl1"        "Ccdc150"        "Coq10b"        
   [81] "Hspe1"          "Mob4"           "Mars2"          "Plcl1"         
   [85] "Maip1"          "Spats2l"        "Sgo2a"          "Aox1"          
   [89] "Aox2"           "Bzw1"           "Nif3l1"         "Ndufb3"        
   [93] "Cflar"          "Casp8"          "Stradb"         "Cdk15"         
   [97] "Fzd7"           "Nop58"          "Bmpr2"          "Fam117b"       
  [101] "Carf"           "Nbeal1"         "Cyp20a1"        "Abi2"          
  [105] "Cd28"           "Ctla4"          "Pard3b"         "Nrp2"          
  [109] "Ino80dos"       "Eef1b2"         "Zdbf2"          "Adam23"        
  [113] "Fastkd2"        "Creb1"          "Ccnyl1"         "Pikfyve"       
  [117] "Pth2r"          "Map2"           "Unc80"          "Rpe"           
  [121] "Spag16"         "Vwc2l"          "Atic"           "Tmem169"       
  [125] "Xrcc5"          "Smarcal1"       "Rpl37a"         "Igfbp2"        
  [129] "Arpc2"          "Gpbar1"         "Pnkd"           "Catip"         
  [133] "Slc11a1"        "Ctdsp1"         "Vil1"           "Plcd4"         
  [137] "Bcs1l"          "Stk36"          "Ttll4"          "Cyp27a1"       
  [141] "Retreg2"        "Zfand2b"        "Ankzf1"         "Stk16"         
  [145] "Dnajb2"         "Speg"           "Gmppa"          "Asic4"         
  [149] "Inha"           "Stk11ip"        "Slc4a3"         "Sgpp2"         
  [153] "Mogat1"         "Acsl3"          "Utp14b"         "Mrpl44"        
  [157] "Nyap2"          "Rhbdd1"         "Col4a3"         "Mff"           
  [161] "Agfg1"          "Daw1"           "Fbxo36"         "Sp100"         
  [165] "Cab39"          "Itm2c"          "Psmd1"          "Armc9"         
  [169] "B3gnt7"         "Ptma"           "Cops7b"         "Dis3l2"        
  [173] "Chrng"          "Eif4e2"         "Efhd1"          "Gigyf2"        
  [177] "Inpp5d"         "Atg16l1"        "Sag"            "Dgkd"          
  [181] "Mroh2a"         "Trpm8"          "Sh3bp4"         "Agap1"         
  [185] "Ackr3"          "Cops8"          "Prlh"           "Lrrfip1"       
  [189] "Ramp1"          "Ube2f"          "Scly"           "Traf3ip1"      
  [193] "Asb1"           "Twist2"         "Gpc1"           "Dusp28"        
  [197] "Rnpepl1"        "Gpr35"          "Crocc2"         "Sned1"         
  [201] "Ppp1r7"         "Ano7"           "Septin2"        "Farp2"         
  [205] "Bok"            "Atg4b"          "Ing5"           "D2hgdh"        
  [209] "Gal3st2b"       "Gal3st2c"       "Neu4"    

In [200]:
nhood_markers = qs::qread('_targets/objects/nhood_markers_ngo_Htr3b')

In [214]:
get_top_milo_degs = function(nhood_markers, subset_groups, n_degs=5, tag){
    subset_groups = stringr::str_split(subset_groups, pattern=fixed('.')) %>% unlist
    p_adj_1 = paste0("adj.P.Val_", subset_groups[1])
    p_adj_2 = paste0("adj.P.Val_", subset_groups[2])
    top_degs = nhood_markers %>% 
        filter(((!!rlang::sym(p_adj_1)) < 0.05) & ((!!rlang::sym(p_adj_2)) < 0.05)) %>%
        mutate(p_product = (!!rlang::sym(p_adj_1)) * (!!rlang::sym(p_adj_2))) %>%
        arrange(p_product) %>%
        head(n_degs) %>%
        pull(GeneID) %>%
        enframe %>%
        rename(gene=value, rank=name) %>%
        mutate(tag = tag)
    top_degs
}

get_top_milo_degs(nhood_markers, 'pos.neg', n_degs=10, tag='Htr3b')

rank,gene,tag
<int>,<chr>,<chr>
1,Trpc5,Htr3b
2,Syn2,Htr3b
3,Cntn3,Htr3b
4,Ppargc1a,Htr3b
5,Nav1,Htr3b
6,Rtl4,Htr3b
7,Fat3,Htr3b
8,Iqsec1,Htr3b
9,Msl1,Htr3b


In [235]:
combined_tmd_ngo = qs::qread('_targets/objects/ngo_panel_5ea')
combined_tmd_ngo %>% dput

c("Mamdc4", "Swap70", "Zfp11", "Ust", "Atf7", "Sorcs2", "Cnot8", 
"Ankra2", "Pcdhb19", "Txnl4b", "Rab11fip5", "Psmd13", "Tcof1", 
"Rab2b", "Smarcc1", "Rdh13", "Tpp2", "Bend7", "Smoc1", "Usp20", 
"Igf1r", "Prkca", "Fbxl5", "Adgb", "Nmt2", "Trpc5", "Syn2", "Cntn3", 
"Ppargc1a", "Nav1", "Zfpm2", "Pias1", "Zfp160", "Abi1", "Atp5b", 
"Zgrf1", "Hat1", "Atad2", "Cdk5rap2", "Gnal", "Fbxw5", "Foxn2", 
"Rpl32", "Arid2", "Kcna1", "Ttll3", "Rock2", "Marchf2", "Thumpd1", 
"Hdac11", "Manbal", "Il1rapl2", "Pcdh15", "Cdc37", "Zdhhc3", 
"R3hcc1", "Txnl4a.1", "Zfp879", "Ccnf", "Atp5e")


In [233]:
get_top_deg_panel = function(combined_tmd_ngo, n_genes){
    panel = combined_tmd_ngo %>%
        filter(rank <= n_genes) %>%
        select(-c(tag, rank)) %>%
        distinct %>%
        pull(gene)
    panel
}



In [234]:
get_top_deg_panel(combined_tmd_ngo, 5)

ERROR: Error in UseMethod("filter"): no applicable method for 'filter' applied to an object of class "character"


In [181]:
subset_groups = 'pos.neg'
n_degs=5

subset_groups = stringr::str_split(subset_groups, pattern=fixed('.')) %>% unlist
p_adj_1 = paste0("adj.P.Val_", subset_groups[1])
p_adj_2 = paste0("adj.P.Val_", subset_groups[2])
top_degs = nhood_markers %>% 
    filter(!!p_adj_1 < 0.05)
#         mutate(p_product = (!!p_adj_1) * (!!p_adj_2)) %>%
#         arrange(p_product) 
top_degs

GeneID,logFC_pos,adj.P.Val_pos,logFC_neg,adj.P.Val_neg
<chr>,<dbl>,<dbl>,<dbl>,<dbl>


In [195]:
nhood_markers %>% filter((!!rlang::sym(p_adj_1)) < 0.05)

GeneID,logFC_pos,adj.P.Val_pos,logFC_neg,adj.P.Val_neg
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Aaas,0.1857544,3.403524e-02,-0.1263514,0.188275228
Aagab,0.6971515,7.306667e-05,-0.5209372,0.005431123
Aamdc,0.4227336,3.989421e-02,-0.2885337,0.199524053
Aars,0.2533034,4.206648e-02,-0.2177834,0.085844229
Abca16,0.3458109,3.144334e-03,-0.2877257,0.014353951
Abcb8,0.4639285,1.470879e-03,-0.3428222,0.027348920
Abce1,0.4140239,1.903176e-02,-0.2574941,0.194355933
Abcf1,0.3806096,3.058151e-03,-0.2678175,0.052668521
Abcf3,0.5340041,1.251453e-04,-0.3971361,0.007748877


In [188]:
p_adj_1

[1] "adj.P.Val_pos"

In [142]:
nhood_markers %>% filter(!!rlang::sym("adj.P.Val_pos") < 0.05)

GeneID,logFC_pos,adj.P.Val_pos,logFC_neg,adj.P.Val_neg
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Ago1,-0.22270038,0.016489610,0.15768451,0.028772750
Bend7,-0.32647824,0.002276320,0.22054993,0.012904788
Cdh18,-0.17623198,0.016489610,0.12569975,0.024305436
Chst9,-0.30312828,0.016489610,0.19530335,0.064867539
Eif4h,-0.26692312,0.021764581,0.17829069,0.066309876
Fbrsl1,0.29297846,0.016489610,-0.20000305,0.051016458
Fmo5,-0.09582427,0.034439954,0.07162831,0.037869363
Ggact,-0.22688867,0.005498155,0.15219351,0.024305436
Gramd3,-0.43561140,0.034439954,0.34853319,0.021463568


In [70]:
milo_obj@nhoodAdjacency

   [[ suppressing 189 column names '2357', '126', '2625' ... ]]



189 x 189 sparse Matrix of class "dgCMatrix"
                                                                               
2357 102  .  .  21  28   9  14  20   4   .  17   .  4  1   5   3   2   2   1  .
126    . 92  1   6   2   8   7   4   1  18   1  24  .  2   .  16   4   7   .  .
2625   .  1 99   7   .   5   2   4   1   3   .  11  .  .   1  10   3   1   1  .
1086  21  6  7 350  55 103  47  20  37  24  73  23  7 20   8  37  27  71   3  8
2082  28  2  .  55 194  18  23  30  14   .  47   3 25  3  18   .   7   5   .  1
1467   9  8  5 103  18 365  68  32  39  51  64  53  1 19  11 100  45  31   3  3
2611  14  7  2  47  23  68 266  96  46  10  61  21 10  .  60  16  21   4  22  2
1806  20  4  4  20  30  32  96 280   7   5  42  11 10  .  37  22  17   .  11  .
1053   4  1  1  37  14  39  46   7 168   2  21   3  1  1  19  11  36  12  11  .
1168   . 18  3  24   .  51  10   5   2 213   6  99  .  9   . 105  20  34   . 15
838   17  1  .  73  47  64  61  42  21   6 183   9  3  2  11   4  12  11   

In [53]:
milo_obj@assays@data$logcounts = milo_obj@assays@data$logcounts[ngo_genes_idx,]
milo_obj@assays@data$logcounts@Dimnames[[1]] = milo_obj@assays@data$logcounts@Dimnames[[1]][ngo_genes_idx]

In [56]:
milo_obj@assays@data$logcounts@Dim

[1] 14396  4023

In [50]:
summary(milo_obj@assays@data$logcounts)

i,j,x
<int>,<int>,<dbl>
3,1,1.3862944
5,1,0.6931472
6,1,4.0775374
8,1,0.6931472
13,1,0.6931472
30,1,0.6931472
35,1,1.0986123
38,1,1.9459101
57,1,0.6931472


In [33]:
s = summary()

[1] 1.3862944 0.6931472 4.0775374 0.6931472 0.6931472 0.6931472 1.0986123
    [8] 1.9459101 0.6931472 0.6931472 1.6094379 0.6931472 0.6931472 0.6931472
   [15] 3.9512437 0.6931472 0.6931472 4.5108595 0.6931472 0.6931472 0.6931472
   [22] 0.6931472 0.6931472 0.6931472 0.6931472 0.6931472 1.0986123 0.6931472
   [29] 0.6931472 0.6931472 0.6931472 1.0986123 0.6931472 0.6931472 1.0986123
   [36] 2.9444390 0.6931472 1.0986123 1.0986123 1.0986123 0.6931472 3.6635616
   [43] 2.0794415 2.5649494 0.6931472 0.6931472 0.6931472 1.0986123 1.7917595
   [50] 0.6931472 1.3862944 0.6931472 0.6931472 0.6931472 0.6931472 0.6931472
   [57] 0.6931472 1.3862944 0.6931472 0.6931472 0.6931472 0.6931472 2.6390573
   [64] 1.3862944 1.0986123 0.6931472 1.3862944 0.6931472 0.6931472 1.3862944
   [71] 2.3978953 1.0986123 0.6931472 0.6931472 1.0986123 0.6931472 1.0986123
   [78] 0.6931472 1.3862944 0.6931472 0.6931472 0.6931472 0.6931472 0.6931472
   [85] 0.6931472 1.0986123 0.6931472 1.0986123 2.1972246 0.6931472 1.0986123
   [92] 0.6931472 0.6931472 0.6931472 0.6931472 0.6931472 1.0986123 0.6931472
   [99] 0.6931472 3.7612001 0.6931472 0.6931472 0.6931472 1.0986123 3.1354942
  [106] 1.3862944 1.0986123 0.6931472 0.6931472 0.6931472 0.6931472 0.6931472
  [113] 1.3862944 0.6931472 1.0986123 0.6931472 2.6390573 2.1972246 0.6931472
  [120] 0.6931472 2.7725887 0.6931472 0.6931472 0.6931472 1.0986123 0.6931472
  [127] 1.0986123 0.6931472 0.6931472 0.6931472 0.6931472 0.6931472 1.6094379
  [134] 0.6931472 2.3978953 0.6931472 0.6931472 1.0986123 0.6931472 0.6931472
  [141] 1.3862944 1.0986123 1.0986123 1.6094379 0.6931472 1.0986123 1.6094379
  [148] 1.0986123 0.6931472 1.6094379 0.6931472 0.6931472 1.7917595 1.0986123
  [155] 0.6931472 0.6931472 0.6931472 0.6931472 0.6931472 0.6931472 0.6931472
  [162] 0.6931472 0.6931472 0.6931472 0.6931472 0.6931472 0.6931472 1.7917595
  [169] 1.3862944 1.3862944 1.3862944 1.3862944 1.7917595 1.0986123 1.0986123
  [176] 0.6931472 0.6931472 1.7917595 0.6931472 1.6094379 0.6931472 0.6931472
  [183] 0.6931472 0.6931472 0.6931472 1.0986123 0.6931472 0.6931472 0.6931472
  [190] 0.6931472 0.6931472 0.6931472 0.6931472 1.0986123 1.0986123 1.9459101
  [197] 0.6931472 0.6931472 0.6931472 0.6931472 1.0986123 0.6931472 0.6931472
  [204] 1.3862944 0.6931472 0.6931472 0.6931472 1.3862944 1.0986123 0.6931472
  [211] 0.6931472 0.6931472 0.6931472 1.0986123 0.6931472 0.6931472 1.0986123
  [218] 0.6931472 0.6931472 0.6931472 1.3862944 0.6931472 1.0986123 0.6931472
  [225] 0.6931472 2.4849066 0.6931472 0.6931472 0.6931472 0.6931472 0.6931472
  [232] 1.3862944 0.6931472 0.6931472 1.3862944 0.6931472 0.6931472 0.6931472
  [239] 1.0986123 0.6931472 0.6931472 1.0986123 0.6931472 0.6931472 0.6931472
  [246] 0.6931472 0.6931472 0.6931472 0.6931472 1.0986123 1.0986123 0.6931472
  [253] 0.6931472 0.6931472 0.6931472 0.6931472 1.6094379 0.6931472 1.6094379
  [260] 0.6931472 0.6931472 0.6931472 1.0986123 1.3862944 0.6931472 0.6931472
  [267] 2.7725887 2.0794415 3.1354942 0.6931472 0.6931472 1.0986123 0.6931472
  [274] 0.6931472 0.6931472 1.0986123 0.6931472 2.0794415 1.6094379 0.6931472
  [281] 0.6931472 1.3862944 1.0986123 0.6931472 0.6931472 0.6931472 0.6931472
  [288] 0.6931472 1.0986123 0.6931472 1.0986123 1.6094379 1.0986123 0.6931472
  [295] 0.6931472 0.6931472 0.6931472 3.1780538 0.6931472 0.6931472 0.6931472
  [302] 0.6931472 0.6931472 1.0986123 1.3862944 0.6931472 1.6094379 0.6931472
  [309] 2.1972246 0.6931472 1.7917595 0.6931472 0.6931472 0.6931472 1.3862944
  [316] 1.0986123 0.6931472 1.0986123 0.6931472 0.6931472 1.3862944 0.6931472
  [323] 0.6931472 0.6931472 0.6931472 1.0986123 0.6931472 0.6931472 0.6931472
  [330] 2.8332133 0.6931472 0.6931472 0.6931472 2.0794415 1.9459101 0.6931472
  [337] 0.6931472 1.0986123 2.3978953 1.6094379 0.6931472 0.6931472 0.6931472
  [344] 0.6931472 0.6931472 1.6094379 0.6931472 0.6931472 1.3862944 0.6931472
  [351] 0.6931472 1.3862944 0.6931472 1.9459101 0.6931472 0.6931472 1.3862944
  [358] 0.6931472 0.693147

In [25]:
milo_obj@assays@data$logcounts[1:10,1:5]

10 x 5 sparse Matrix of class "dgCMatrix"
                   ATTACTCTCCGCGGAT_1_1 AGTTCGAAGAAACCAT_1_1
Lypla1                        .                    .        
Tcea1                         .                    0.6931472
Atp6v1h                       1.3862944            0.6931472
Oprk1                         .                    .        
Rb1cc1                        0.6931472            1.0986123
St18                          4.0775374            4.1896547
Pcmtd1                        .                    0.6931472
Rrs1                          0.6931472            .        
Adhfe1                        .                    .        
ENSMUSG00000097893            .                    .        
                   CACGGGTGTTACACTG_1_1 GCCAACGGTTGTTTGG_1_1
Lypla1                        .                    0.6931472
Tcea1                         0.6931472            0.6931472
Atp6v1h                       .                    0.6931472
Oprk1                         .            

In [229]:
exp_ngo_all = qs::qread('../genebasisr/_targets/objects/exp_ngo_all')
exp_ngo_all

An object of class Seurat 
38054 features across 144643 samples within 2 assays 
Active assay: SCT (18696 features, 3000 variable features)
 1 other assay present: RNA
 2 dimensional reductions calculated: pca, umap

In [230]:
144643/2

[1] 72321.5

In [236]:
1

[1] 1

In [237]:
combined_da_results_obobBL6d5 = qs::qread('_targets/objects/combined_da_results_obobBL6d5')

In [251]:
combined_da_results_obobBL6d5 %>%
filter(SpatialFDR < 0.1) %>%
dim

combined_da_results_obob5v5 %>% dim

[1] 1634   11

[1] 9329   11

In [241]:
combined_da_results_obob5v5 = qs::qread('_targets/objects/combined_da_results_obob5v5')

In [316]:
combined_da_results_obob5v5 %>% head

,logFC,logCPM,F,PValue,FDR,Nhood,SpatialFDR,polarity,labels,labels_fraction,n_cells
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
1,5.175528,12.75419,65.87486,2.712965e-11,6.592504e-09,63,6.856218e-09,pos,a1_Tany__Astrocytes,1,129
2,4.883802,12.97953,50.94093,1.324004e-09,1.608665e-07,93,1.592806e-07,pos,a1_Tany__Astrocytes,1,166
3,3.576113,12.47521,33.10595,2.990955e-07,2.422673e-05,214,2.576412e-05,pos,a1_Tany__Astrocytes,1,99
4,3.401746,12.73492,31.07434,5.936490e-07,3.606418e-05,118,3.781441e-05,pos,a1_Tany__Astrocytes,1,135
5,3.177160,13.10128,23.56111,8.712993e-06,4.234514e-04,19,4.260118e-04,pos,a1_Tany__Astrocytes,1,201
6,2.661283,13.13157,21.17582,2.161802e-05,8.755300e-04,58,8.618807e-04,pos,a1_Tany__Astrocytes,1,212


In [337]:
combined_da_results_obobFGF1BL6d5 = qs::qread('_targets/objects/combined_da_results_obobFGF1BL6d5')
combined_da_results_obobFGF1BL6d5 %>% head

,logFC,logCPM,F,PValue,FDR,Nhood,SpatialFDR,polarity,labels,labels_fraction,n_cells
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
1,5.017764,12.75419,36.14085,1.105710e-07,2.686875e-05,63,2.794356e-05,pos,a1_Tany__Astrocytes,1,129
2,5.447913,12.47521,31.82251,4.603370e-07,5.593094e-05,214,6.346384e-05,pos,a1_Tany__Astrocytes,1,99
3,4.304857,12.94578,27.74275,1.894365e-06,1.534436e-04,108,1.575240e-04,pos,a1_Tany__Astrocytes,1,174
4,3.878225,12.79742,26.79549,2.657774e-06,1.614598e-04,165,1.604885e-04,pos,a1_Tany__Astrocytes,1,152
5,3.320485,12.97953,19.40802,4.323617e-05,2.101278e-03,93,2.067135e-03,pos,a1_Tany__Astrocytes,1,166
6,-4.049815,12.24034,17.56872,9.065023e-05,2.753501e-03,25,2.686801e-03,neg,a1_Tany__Astrocytes,1,89


In [250]:
combined_da_results_obob5v5 %>%
filter(SpatialFDR < 0.1) %>%
dim

combined_da_results_obobBL6d5 %>% dim

[1] 842  11

[1] 9329   11

In [338]:
obob5v5 = combined_da_results_obob5v5 %>%
select(labels, polarity, Nhood, FDR, n_cells)

obobBL6d5 = combined_da_results_obobBL6d5 %>%
select(labels, polarity, Nhood, FDR, n_cells)

obobFGF1BL6d5 = combined_da_results_obobFGF1BL6d5 %>%
select(labels, polarity, Nhood, FDR, n_cells)



In [352]:
obobFGF1BL6d5 %>% head

,labels,polarity,Nhood,FDR,n_cells
,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,a1_Tany__Astrocytes,pos,63,2.686875e-05,129
2,a1_Tany__Astrocytes,pos,214,5.593094e-05,99
3,a1_Tany__Astrocytes,pos,108,1.534436e-04,174
4,a1_Tany__Astrocytes,pos,165,1.614598e-04,152
5,a1_Tany__Astrocytes,pos,93,2.101278e-03,166
6,a1_Tany__Astrocytes,neg,25,2.753501e-03,89


In [354]:
avb = left_join(obob5v5, obobBL6d5, by = c("labels", "Nhood"))
avb = left_join(avb, obobFGF1BL6d5, by = c("labels", "Nhood"))

bl6_restored = avb %>% 
filter((FDR.x < 0.1) & (FDR.y < 0.1)) %>% 
filter((polarity.x != polarity.y)) %>%
group_by(labels) %>%
summarise(n_restored = n()) %>% arrange(desc(n_restored))

bl6_restored_strict = avb %>% 
filter((FDR.x < 0.1) & (FDR.y < 0.1) & (FDR >= 0.1)) %>% 
filter((polarity.x != polarity.y)) %>%
group_by(labels) %>%
summarise(n_restored = n()) %>% arrange(desc(n_restored))

total_summary = avb %>%
group_by(labels) %>%
summarise(n_total = n())

fgf1_summary = avb %>%
group_by(labels) %>%
filter(FDR.x < 0.1) %>%
summarise(n_fgf1 = n())

bl6_summary = avb %>%
group_by(labels) %>%
filter(FDR.y < 0.1) %>%
summarise(n_BL6 = n())

total_summary = total_summary %>% 
full_join(fgf1_summary) %>%
full_join(bl6_summary) %>%
mutate(n_fgf1 = replace_na(n_fgf1, 0)) %>%
mutate(n_BL6 = replace_na(n_BL6, 0))
# total_summary

total_summary = left_join(total_summary, bl6_restored) %>%
mutate(n_restored = replace_na(n_restored, 0)) %>%
mutate(prop__restored_BL6 = n_restored/n_BL6) %>%
mutate(prop__restored_FGF1 = n_restored/n_fgf1) %>%
arrange(desc(n_fgf1))
total_summary


Joining, by = "labels"
Joining, by = "labels"
Joining, by = "labels"


labels,n_total,n_fgf1,n_BL6,n_restored,prop__restored_BL6,prop__restored_FGF1
<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>
Astrocytes,607,198,291,117,0.40206186,0.59090909
Htr3b,244,75,43,8,0.18604651,0.10666667
a1_Tany__Astrocytes,243,60,50,15,0.30000000,0.25000000
MOL,189,60,119,48,0.40336134,0.80000000
b1_Tanycytes,233,59,11,2,0.18181818,0.03389831
Trh_Lef1,214,53,7,3,0.42857143,0.05660377
Agrp,316,52,249,35,0.14056225,0.67307692
NG2_OPC2,90,43,35,27,0.77142857,0.62790698
Sst_Unc13c__Agrp,202,37,122,27,0.22131148,0.72972973


In [361]:
bl6_restored %>% left_join(bl6_restored_strict, by = c("labels"), suffix = c("", ".strict"))

labels,n_restored,n_restored.strict
<chr>,<int>,<int>
Astrocytes,117,98
MOL,48,27
Agrp,35,16
NG2_OPC2,27,22
Sst_Unc13c__Agrp,27,14
a1_Tany__Astrocytes,15,14
Ghrh,10,1
Unassigned1.sc17.sc28,10,10
Pomc_Glipr1__Slc17a6_Trhr,9,8


In [356]:
bl6_restored

labels,n_restored
<chr>,<int>
Astrocytes,117
MOL,48
Agrp,35
NG2_OPC2,27
Sst_Unc13c__Agrp,27
a1_Tany__Astrocytes,15
Ghrh,10
Unassigned1.sc17.sc28,10
Pomc_Glipr1__Slc17a6_Trhr,9


In [ ]:
options(width=120)
total_summary %>% select(-prop__restored_BL6) %>% print(width=120, n=200)

In [274]:
avb %>% 
group_by(labels) %>%
summarise(n = n()) %>% arrange(desc(n))

labels,n
<chr>,<int>
Nr5a1_Bdnf,1407
Unassigned2,831
Astrocytes,607
Agrp,316
Slc17a6_Fam19a2,303
Rgs16_Nmu,297
Tmem215,286
Unassigned1.sc17.sc28,265
Trh_Cxcl12,251
